In [1]:
import emoji # pip install emoji
import re
import os
import sys
import time
import string
import json
import pandas as pd
import csv
import sklearn

from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

I believe you need the your training graph here

In [2]:
f = open("myfile.json", "r", encoding='utf-8')
sl = json.load(f)
f.close()

In [3]:
all_word_set = {}
for key in sl.keys():
    for word in sl[key].keys():
        if word not in all_word_set:
            all_word_set[word] = 1
        else:
            all_word_set[word] += 1

load in the test dataset(I have generate it for you)

In [4]:
test_df = pd.read_csv('test_rec.csv')

filter the dataset with `all_word_set` and `all_emoji`

In [5]:
def filter_test_data(all_word_set, all_emoji, test_df):
    test_df = test_df.drop(labels = list(test_df[test_df.apply(lambda row:(not row.word in all_word_set) or (not row.emoji in all_emoji),axis=1)].index), axis=0)
    return test_df

In [6]:
test_df = filter_test_data(all_word_set, sl.keys(), test_df)

In [7]:
test_df

,word,emoji
1,"you,",:red_heart_selector:
2,love,:red_heart_selector:
3,hearts,:red_heart_selector:
4,you.,:red_heart_selector:
5,linkedin,:red_heart_selector:
...,...,...
6834,love,:kissing_face_with_closed_eyes:
6838,h,:keycap_7:
6839,k,:keycap_7:
6840,r,:keycap_7:


Evaluation function

In [14]:
def evaluate(test_df, predict_func):
    test_df['res'] = test_df.apply(lambda row:predict_func(row.word, row.emoji) ,axis=1)
    return test_df.mean().res

random predictor

In [15]:
# random predictor
def random_pred(word, emoji):
    global sl # can be removed
    return 1/len(sl)

In [16]:
evaluate(test_df, random_pred)

0.0007639419404124907

trivial neighbor method

In [17]:
# trivial_neighbor predictor
def trivial_neighbor(word, emoji):
    global sl
    total_weight = 0
    good_weight = 0
    for key in sl.keys():
        if word in sl[key].keys():
            total_weight += sl[key][word]
            if key == emoji:
                good_weight += sl[key][word]
    if total_weight == 0:
        return 0
    else:
        return good_weight/total_weight

In [18]:
evaluate(test_df, trivial_neighbor)

0.04321099372389337

## Projection

In [8]:
def evaluate_projection(test_df, predict_func):
    test_df['res'] = test_df.apply(lambda row:predict_func(row.word, row.emoji, FG, Gp1) ,axis=1)
    return test_df.mean().res

In [9]:
import projection_model as pm
FG, Gp1 = pm.load_graph("Total_FG.p", "Sample_GP.p")

In [10]:
word_in = 'launch'
emoji_in = ':rocket:'
# word_in = '(profits'
# emoji_in = ':rocket:'

In [11]:
pm.projection_method(word_in, emoji_in, FG, Gp1)

0.33760459891884886

In [13]:
evaluate_projection(test_df, pm.projection_method)

0.037352181568767365

In [19]:
evaluate(test_df, trivial_neighbor)

0.04321099372389337